# Blockchain introduction seminar

Based on: 
* https://github.com/santisiri/proof-of-work/blob/master/pow.py
* http://ecomunsing.com/build-your-own-blockchain
* https://github.com/JaeDukSeo/Simple-Merkle-Tree-in-Python
* https://www.laurentluce.com/posts/python-and-cryptography-with-pycrypto/

## 1. Proof-of-Work

This is a simple implemetation of Proof-of-Work function.
It takes current block header and block calculation difficulty as inputs and returns nonce for this block and the header of a mined block.

In [1]:
import hashlib

def proof_of_work(header, difficulty_bits):
    max_nonce = 2 ** 32
    target = 2 ** (256-difficulty_bits)
    for nonce in range(max_nonce):
        hash_result = hashlib.sha256(str(header).encode('utf-8')+str(nonce).encode('utf-8')).hexdigest()

        if int(hash_result, 16) < target:
            return (hash_result, nonce)

The root block is calulated here.

In [2]:
import time

nonce = 0
block_hash = ""
new_block = 'This is the root block' + block_hash

difficulty_bits = 16

start_time = time.time()
(block_hash, nonce) = proof_of_work(new_block, difficulty_bits)
end_time = time.time()
elapsed_time = end_time - start_time

print("Success with nonce ", nonce)
print("Hash is ", block_hash)
print("Elapsed time: %.4f seconds" % elapsed_time)

Success with nonce  2719
Hash is  00000a951c4f8a88713ec38209093a852463fd07023b7c07b1d150bac521a9b3
Elapsed time: 0.0120 seconds


### Task 1.1

Calculate N=100 blocks for different difficulty_bits. Plot mean, mean+std and mean-std for log_2(elapsed_time) and log_2(nonce+1) as a function of difficulty_bits. 

What kind of functions do you expect to see theortically?

(*) Use linear regression to fit linear model. What slope do you get? How to interpret the shift?
What hashrate does you computer performs?

*Note.* Each next block should be calculated using the hash of the previous one.

In [11]:
# code here
import numpy as np
difficulty_bits_max=16
number_of_experiments=100
estimated_times=np.zeros((number_of_experiments,difficulty_bits_max))
for difficulty_bits in range(16):
    print(difficulty_bits)
    for index in range(number_of_experiments):
        new_block=block_hash
        start_time = time.time()
        (block_hash, nonce) = proof_of_work(new_block, difficulty_bits)
        end_time = time.time()
        elapsed_time[index,d] = end_time - start_time
    



0
1
2
3
4
5
6
7
8
9
10
11
12
13


KeyboardInterrupt: 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.semilogy(range(difficulty_bits_max),np.mean)

### Task 1.2

Change proof_of_work fuction to get a target parameter insted of difficulty_bits.

What are the resonable limitations for the target?

Choose such a target that you will calculated 20 160 000 blocks per 14 days in avarage. Prove it experimetally (estimate time for at least 2016 blocks and show that it is close to the goal).

(*) Use Binomial random model to estimate the mean time for block and its confidance.

In [ ]:
# code here

### Task 1.3

Repeat experiments for other hash functions (for example, md5(), sha1(), sha224(), sha384(), and sha512()).

What changes do you expect and what changes do you see?

In [ ]:
# code here

## 2. Simple cryptocurrancy

We’ll start off by tracking the accounts of two imaginary people: Alice and Bob, who will trade virtual money with each other.
We’ll need to create a transaction pool of incoming transactions, validate those transactions, and make them into a block.
We’ll be using a hash function to create a ‘fingerprint’ for each of our transactions.

In [12]:
import hashlib, json, sys

def hashMe(msg=""):
    # For convenience, this is a helper function that wraps our hashing algorithm
    if type(msg)!=str:
        msg = json.dumps(msg,sort_keys=True)  # If we don't sort keys, we can't guarantee repeatability!
        
    if sys.version_info.major == 2:
        return unicode(hashlib.sha256(msg).hexdigest(),'utf-8')
    else:
        return hashlib.sha256(str(msg).encode('utf-8')).hexdigest()

Next, we want to create a function to generate exchanges between Alice and Bob. We’ll indicate withdrawals with negative numbers, and deposits with positive numbers. We’ll construct our transactions to always be between the two users of our system, and make sure that the deposit is the same magnitude as the withdrawal- i.e. that we’re neither creating nor destroying money.

In [13]:
import random

def makeTransaction(maxValue=3):
    # This will create valid transactions in the range of (1,maxValue)
    sign      = int(random.getrandbits(1))*2 - 1   # This will randomly choose -1 or 1
    amount    = random.randint(1,maxValue)
    alicePays = sign * amount
    bobPays   = -1 * alicePays
    # By construction, this will always return transactions that respect the conservation of tokens.
    # However, note that we have not done anything to check whether these overdraft an account
    return {u'Alice':alicePays,u'Bob':bobPays}

Now let’s create a large set of transactions, then chunk them into blocks.

In [15]:
txnBuffer = [makeTransaction() for i in range(30)]
print(txnBuffer)

[{'Alice': -1, 'Bob': 1}, {'Alice': 3, 'Bob': -3}, {'Alice': -2, 'Bob': 2}, {'Alice': 1, 'Bob': -1}, {'Alice': -3, 'Bob': 3}, {'Alice': 1, 'Bob': -1}, {'Alice': -2, 'Bob': 2}, {'Alice': -3, 'Bob': 3}, {'Alice': 3, 'Bob': -3}, {'Alice': 3, 'Bob': -3}, {'Alice': -2, 'Bob': 2}, {'Alice': -1, 'Bob': 1}, {'Alice': -2, 'Bob': 2}, {'Alice': 1, 'Bob': -1}, {'Alice': -2, 'Bob': 2}, {'Alice': -2, 'Bob': 2}, {'Alice': 3, 'Bob': -3}, {'Alice': 2, 'Bob': -2}, {'Alice': 3, 'Bob': -3}, {'Alice': -3, 'Bob': 3}, {'Alice': 1, 'Bob': -1}, {'Alice': -3, 'Bob': 3}, {'Alice': -1, 'Bob': 1}, {'Alice': 1, 'Bob': -1}, {'Alice': 3, 'Bob': -3}, {'Alice': -2, 'Bob': 2}, {'Alice': -1, 'Bob': 1}, {'Alice': 2, 'Bob': -2}, {'Alice': -3, 'Bob': 3}, {'Alice': 3, 'Bob': -3}]


For Bitcoin, the validation function checks that the input values are valid unspent transaction outputs (UTXOs), that the outputs of the transaction are no greater than the input, and that the keys used for the signatures are valid. In Ethereum, the validation function checks that the smart contracts were faithfully executed and respect gas limits.

No worries, though- we don’t have to build a system that complicated. We’ll define our own, very simple set of rules which make sense for a basic token system:

* The sum of deposits and withdrawals must be 0 (tokens are neither created nor destroyed)
* A user’s account must have sufficient funds to cover any withdrawals

If either of these conditions are violated, we’ll reject the transaction.

In [16]:
def updateState(txn, state):
    # Inputs: txn, state: dictionaries keyed with account names, holding numeric values for transfer amount (txn) or account balance (state)
    # Returns: Updated state, with additional users added to state if necessary
    # NOTE: This does not not validate the transaction- just updates the state!
    
    # If the transaction is valid, then update the state
    state = state.copy() # As dictionaries are mutable, let's avoid any confusion by creating a working copy of the data.
    for key in txn:
        if key in state.keys():
            state[key] += txn[key]
        else:
            state[key] = txn[key]
    return state

In [17]:
def isValidTxn(txn,state):
    # Assume that the transaction is a dictionary keyed by account names

    # Check that the sum of the deposits and withdrawals is 0
    if sum(txn.values()) is not 0:
        return False
    
    # Check that the transaction does not cause an overdraft
    for key in txn.keys():
        if key in state.keys(): 
            acctBalance = state[key]
        else:
            acctBalance = 0
        if (acctBalance + txn[key]) < 0:
            return False
    
    return True

Here are a set of sample transactions, some of which are fraudulent - but we can now check their validity!

In [18]:
state = {u'Alice':5,u'Bob':5}
print(isValidTxn({u'Alice': -3, u'Bob': 3},state))  # Basic transaction - this works great!
print(isValidTxn({u'Alice': -4, u'Bob': 3},state))  # But we can't create or destroy tokens!
print(isValidTxn({u'Alice': -6, u'Bob': 6},state))  # We also can't overdraft our account.
print(isValidTxn({u'Alice': -4, u'Bob': 2,'Lisa':2},state)) # Creating new users is valid
print(isValidTxn({u'Alice': -4, u'Bob': 3,'Lisa':2},state)) # But the same rules still apply!


True
False
False
True
False


### Building the Blockchain: From Transactions to Blocks

In [19]:
state = {u'Alice':50, u'Bob':50}  # Define the initial state
genesisBlockTxns = [state]
genesisBlockContents = {u'blockNumber':0,u'parentHash':None,u'txnCount':1,u'txns':genesisBlockTxns}
genesisHash = hashMe( genesisBlockContents )
genesisBlock = {u'hash':genesisHash,u'contents':genesisBlockContents}
genesisBlockStr = json.dumps(genesisBlock, sort_keys=True)

In [20]:
chain = [genesisBlock]

In [21]:
def makeBlock(txns,chain):
    parentBlock = chain[-1]
    parentHash  = parentBlock[u'hash']
    blockNumber = parentBlock[u'contents'][u'blockNumber'] + 1
    txnCount    = len(txns)
    blockContents = {u'blockNumber':blockNumber,u'parentHash':parentHash,
                     u'txnCount':len(txns),'txns':txns}
    blockHash = hashMe( blockContents )
    block = {u'hash':blockHash,u'contents':blockContents}
    
    return block

Let’s use this to process our transaction buffer into a set of blocks:

In [22]:
blockSizeLimit = 5  # Arbitrary number of transactions per block- 
               #  this is chosen by the block miner, and can vary between blocks!

while len(txnBuffer) > 0:
    bufferStartSize = len(txnBuffer)
    
    ## Gather a set of valid transactions for inclusion
    txnList = []
    while (len(txnBuffer) > 0) & (len(txnList) < blockSizeLimit):
        newTxn = txnBuffer.pop()
        validTxn = isValidTxn(newTxn,state) # This will return False if txn is invalid
        
        if validTxn:           # If we got a valid state, not 'False'
            txnList.append(newTxn)
            state = updateState(newTxn,state)
        else:
            print("ignored transaction")
            sys.stdout.flush()
            continue  # This was an invalid transaction; ignore it and move on
        
    ## Make a block
    myBlock = makeBlock(txnList,chain)
    chain.append(myBlock)  

In [23]:
chain[0]

{'hash': '7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507',
 'contents': {'blockNumber': 0,
  'parentHash': None,
  'txnCount': 1,
  'txns': [{'Alice': 50, 'Bob': 50}]}}

In [24]:
chain[1]

{'hash': 'ccceb96e35459ff98c49a2607ec0929a422f8ad025a5fe1fde04c5e6d3e74aa0',
 'contents': {'blockNumber': 1,
  'parentHash': '7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507',
  'txnCount': 5,
  'txns': [{'Alice': 3, 'Bob': -3},
   {'Alice': -3, 'Bob': 3},
   {'Alice': 2, 'Bob': -2},
   {'Alice': -1, 'Bob': 1},
   {'Alice': -2, 'Bob': 2}]}}

In [25]:
print(len(chain))

7


As expected, the genesis block includes an invalid transaction which initiates account balances (creating tokens out of thin air). The hash of the parent block is referenced in the child block, which contains a set of new transactions which affect system state. We can now see the state of the system, updated to include the transactions:

In [26]:
state

{'Alice': 47, 'Bob': 53}

### Checking Chain Validity

In [27]:
def checkBlockHash(block):
    # Raise an exception if the hash does not match the block contents
    expectedHash = hashMe( block['contents'] )
    if block['hash']!=expectedHash:
        raise Exception('Hash does not match contents of block %s'%
                        block['contents']['blockNumber'])
    return

In [28]:
def checkBlockValidity(block,parent,state):    
    # We want to check the following conditions:
    # - Each of the transactions are valid updates to the system state
    # - Block hash is valid for the block contents
    # - Block number increments the parent block number by 1
    # - Accurately references the parent block's hash
    parentNumber = parent['contents']['blockNumber']
    parentHash   = parent['hash']
    blockNumber  = block['contents']['blockNumber']
    
    # Check transaction validity; throw an error if an invalid transaction was found.
    for txn in block['contents']['txns']:
        if isValidTxn(txn,state):
            state = updateState(txn,state)
        else:
            raise Exception('Invalid transaction in block %s: %s'%(blockNumber,txn))

    checkBlockHash(block) # Check hash integrity; raises error if inaccurate

    if blockNumber!=(parentNumber+1):
        raise Exception('Hash does not match contents of block %s'%blockNumber)

    if block['contents']['parentHash'] != parentHash:
        raise Exception('Parent hash not accurate at block %s'%blockNumber)
    
    return state

In [29]:
def checkChain(chain):
    # Work through the chain from the genesis block (which gets special treatment), 
    #  checking that all transactions are internally valid,
    #    that the transactions do not cause an overdraft,
    #    and that the blocks are linked by their hashes.
    # This returns the state as a dictionary of accounts and balances,
    #   or returns False if an error was detected

    
    ## Data input processing: Make sure that our chain is a list of dicts
    if type(chain)==str:
        try:
            chain = json.loads(chain)
            assert( type(chain)==list)
        except:  # This is a catch-all, admittedly crude
            return False
    elif type(chain)!=list:
        return False
    
    state = {}
    ## Prime the pump by checking the genesis block
    # We want to check the following conditions:
    # - Each of the transactions are valid updates to the system state
    # - Block hash is valid for the block contents

    for txn in chain[0]['contents']['txns']:
        state = updateState(txn,state)
    checkBlockHash(chain[0])
    parent = chain[0]
    
    ## Checking subsequent blocks: These additionally need to check
    #    - the reference to the parent block's hash
    #    - the validity of the block number
    for block in chain[1:]:
        state = checkBlockValidity(block,parent,state)
        parent = block
        
    return state

We can now check the validity of the state:

In [30]:
checkChain(chain)

{'Alice': 47, 'Bob': 53}

And even if we are loading the chain from a text file, e.g. from backup or loading it for the first time, we can check the integrity of the chain and create the current state:

In [31]:
chainAsText = json.dumps(chain,sort_keys=True)
checkChain(chainAsText)

{'Alice': 47, 'Bob': 53}

In [32]:
import copy
nodeBchain = copy.copy(chain)
nodeBtxns  = [makeTransaction() for i in range(5)]
newBlock   = makeBlock(nodeBtxns,nodeBchain)

In [33]:
print("Blockchain on Node A is currently %s blocks long"%len(chain))

try:
    print("New Block Received; checking validity...")
    state = checkBlockValidity(newBlock,chain[-1],state) # Update the state- this will throw an error if the block is invalid!
    chain.append(newBlock)
except:
    print("Invalid block; ignoring and waiting for the next block...")

print("Blockchain on Node A is now %s blocks long"%len(chain))

Blockchain on Node A is currently 7 blocks long
New Block Received; checking validity...
Blockchain on Node A is now 8 blocks long


### Task 2.1

Add Proof-of-Work to the blockchain and nonce to the block header.

### Task 2.2

Add nonce field to the transactions and include into the transaction check function to avoid transaction repeat.

## 3. Merkle tree

In [34]:
# 0. Import the needed library
import hashlib,json
from collections import OrderedDict

# 1. Declare the class trees
class Jae_MerkTree:

    # 2. Initiate the class object
    def __init__(self,listoftransaction=None):
        self.listoftransaction = listoftransaction
        self.past_transaction = OrderedDict()

    # 3. Create the Merkle Tree  
    def create_tree(self):

        # 3.0 Continue on the declaration
        listoftransaction = self.listoftransaction
        past_transaction = self.past_transaction
        temp_transaction = []

        # 3.1 Loop until the list finishes
        for index in range(0,len(listoftransaction),2):

            # 3.2 Get the most left element 
            current = listoftransaction[index]

            # 3.3 If there is still index left get the right of the left most element
            if index+1 != len(listoftransaction):
                current_right = listoftransaction[index+1]

            # 3.4 If we reached the limit of the list then make a empty string
            else:
                current_right = ''

            # 3.5 Apply the Hash 256 function to the current values
            current_hash = hashlib.sha256(current.encode('utf-8'))

            # 3.6 If the current right hash is not a '' <- empty string
            if current_right != '':
                current_right_hash = hashlib.sha256(current_right.encode('utf-8'))

            # 3.7 Add the Transaction to the dictionary 
            past_transaction[listoftransaction[index]] = current_hash.hexdigest()

            # 3.8 If the next right is not empty
            if current_right != '':
                past_transaction[listoftransaction[index+1]] = current_right_hash.hexdigest()

            # 3.9 Create the new list of transaction
            if current_right != '':
                temp_transaction.append(current_hash.hexdigest() + current_right_hash.hexdigest())

            # 3.01 If the left most is an empty string then only add the current value
            else:
                temp_transaction.append(current_hash.hexdigest())

        # 3.02 Update the variables and rerun the function again 
        if len(listoftransaction) != 1:
            self.listoftransaction = temp_transaction
            self.past_transaction = past_transaction

            # 3.03 Call the function repeatly again and again until we get the root 
            self.create_tree()

    # 4. Return the past Transaction 
    def Get_past_transacion(self):
        return self.past_transaction

    # 5. Get the root of the transaction
    def Get_Root_leaf(self):
        last_key = list(self.past_transaction.keys())[-1]
        return self.past_transaction[last_key]

# Declare the main part of the function to run
if __name__ == "__main__":

    # a) Create the new class of Jae_MerkTree
    Jae_Tree = Jae_MerkTree()

    # b) Give list of transaction
    transaction = ['a','b','c','d']

    # c) pass on the transaction list 
    Jae_Tree.listoftransaction = transaction

    # d) Create the Merkle Tree transaction
    Jae_Tree.create_tree()

    # e) Retrieve the transaction 
    past_transaction = Jae_Tree.Get_past_transacion()

    # f) Get the last transaction and print all 
    print("First Example - Even number of transaction Merkel Tree")
    print('Final root of the tree : ',Jae_Tree.Get_Root_leaf())
    print(json.dumps(past_transaction, indent=4))
    print("-" * 50)

    # h) Second example
    print("Second Example - Odd number of transaction Merkel Tree")
    Jae_Tree = Jae_MerkTree()
    transaction = ['a','b','c','d','e']
    Jae_Tree.listoftransaction = transaction
    Jae_Tree.create_tree()
    past_transaction = Jae_Tree.Get_past_transacion()
    print('Final root of the tree : ',Jae_Tree.Get_Root_leaf())
    print(json.dumps(past_transaction, indent=4))
    print("-" * 50)

    # i) Actual Use Case
    print("Final Example - Actuall use case of the Merkle Tree")

    # i-1) Declare a transaction - the ground truth
    ground_truth_Tree = Jae_MerkTree()
    ground_truth_transaction = ['a','b','c','d','e']
    ground_truth_Tree.listoftransaction = ground_truth_transaction
    ground_truth_Tree.create_tree()
    ground_truth_past_transaction = ground_truth_Tree.Get_past_transacion()
    ground_truth_root = ground_truth_Tree.Get_Root_leaf()

    # i-2) Declare a tampered transaction
    tampered_Tree = Jae_MerkTree()
    tampered_Tree_transaction = ['a','b','c','d','f']
    tampered_Tree.listoftransaction = tampered_Tree_transaction
    tampered_Tree.create_tree()
    tampered_Tree_past_transaction = tampered_Tree.Get_past_transacion()
    tampered_Tree_root = tampered_Tree.Get_Root_leaf()

    # i-3) The three company share all of the transaction 
    print('Company A - my final transaction hash : ',ground_truth_root)
    print('Company B - my final transaction hash : ',ground_truth_root)
    print('Company C - my final transaction hash : ',tampered_Tree_root)

    # i-4) Print out all of the past transaction
    print("\n\nGround Truth past Transaction ")
    print(json.dumps(ground_truth_past_transaction, indent=4))
    
    print("\n\nTamper Truth past Transaction ")
    print(json.dumps(tampered_Tree_past_transaction, indent=4))

First Example - Even number of transaction Merkel Tree
Final root of the tree :  58c89d709329eb37285837b042ab6ff72c7c8f74de0446b091b6a0131c102cfd
{
    "a": "ca978112ca1bbdcafac231b39a23dc4da786eff8147c4e72b9807785afee48bb",
    "b": "3e23e8160039594a33894f6564e1b1348bbd7a0088d42c4acb73eeaed59c009d",
    "c": "2e7d2c03a9507ae265ecf5b5356885a53393a2029d241394997265a1a25aefc6",
    "d": "18ac3e7343f016890c510e93f935261169d9e3f565436429830faf0934f4f8e4",
    "ca978112ca1bbdcafac231b39a23dc4da786eff8147c4e72b9807785afee48bb3e23e8160039594a33894f6564e1b1348bbd7a0088d42c4acb73eeaed59c009d": "62af5c3cb8da3e4f25061e829ebeea5c7513c54949115b1acc225930a90154da",
    "2e7d2c03a9507ae265ecf5b5356885a53393a2029d241394997265a1a25aefc618ac3e7343f016890c510e93f935261169d9e3f565436429830faf0934f4f8e4": "d3a0f1c792ccf7f1708d5422696263e35755a86917ea76ef9242bd4a8cf4891a",
    "62af5c3cb8da3e4f25061e829ebeea5c7513c54949115b1acc225930a90154dad3a0f1c792ccf7f1708d5422696263e35755a86917ea76ef9242bd4a8cf4891a": 

### Task 3.1

Replace transactions list with their Merkle root it the block header.

## Public key algorithms

In [44]:
from Crypto import Random
from Crypto.Hash import MD5
from Crypto.PublicKey import RSA
random_generator = Random.new().read
key = RSA.generate(1024, random_generator)
print(key)

ImportError: cannot import name 'winrandom'

In [40]:
print(key.can_encrypt())
print(key.can_sign())
print(key.has_private())

NameError: name 'key' is not defined

### Encrypt

In [41]:
public_key = key.publickey()
enc_data = public_key.encrypt(str('abcdefgh').encode('utf-8'), 32)
print(enc_data)

NameError: name 'key' is not defined

### Decrypt

In [ ]:
key.decrypt(enc_data)

### Sign

In [ ]:
from Crypto.Hash import MD5
from Crypto.PublicKey import RSA
RSAkey = RSA.generate(1024, random_generator)   # This will take a while...
text = 'abcdefgh'
plaintext = str(text).encode('utf-8')
hash = MD5.new(plaintext).digest()
signature = RSAkey.sign(hash, "")
print(signature)

### Verify

In [ ]:
print(RSAkey.verify(hash, signature))     # This sig will check out
print(RSAkey.verify(hash[:-1], signature)) # This sig will fail

### Task 4.1

Add signatures to the blockchain and their verification in isValidTxn.

### Task 4.2

Prepare demo for a resulting blockchain.

## Task  5

1. Make amounts int32. Be aware of overflows!

2. Add lock_time parameter into transactions with the following logic: one can't add it into the block before height given in lock_time.

3. (\*) Implement multisig addresses https://en.bitcoin.it/wiki/Multisignature

4. (\*) Implement UTXOs https://en.wikipedia.org/wiki/Unspent_transaction_output

5. (\*) Prepare demo with micropayment channel https://bitcoin.org/en/developer-guide#micropayment-channel